# Imports

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment

# Requests

In [2]:
team_abbrev = "dal"
season = "2022"
url = f"https://www.pro-football-reference.com/teams/{team_abbrev}/{season}.htm"

In [3]:
r = requests.get(url)

In [4]:
r.status_code

200

# Soup to Table

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')

In [6]:
target_ids = ['team_stats', 'games']
table_list = list()
for table_id in target_ids:
    table = soup.select_one(f"table#{table_id}")
    if table.get("id") == table_id:
        table_list.append(table)

# Building Team Stat df

In [7]:
team_stat_table = table_list[0]
rows = team_stat_table.find_all("tr")

team_stat_rows = list()
for i, row in enumerate(rows[1:]):
    if i == 0:
        header_text = [td.get_text(strip=True) for td in row.find_all(['th', 'td'])]
        for i in range(len(header_text)):
            if 1 <= i < 8:
                header_text[i] = f"{header_text[i]}_totals"
            elif 8 <= i < 15:
                header_text[i] = f"{header_text[i]}_passing_totals"
            elif 15 <= i < 20:
                header_text[i] = f"{header_text[i]}_rushing_totals"
            elif 20 <= i < 23:
                header_text[i] = f"{header_text[i]}_penalties"
            elif 23 <= i < 26:
                header_text[i] = f"{header_text[i]}_drives"
            elif 26 <= i:
                header_text[i] = f"{header_text[i]}_average_drive"
        
        team_stat_rows.append(header_text)
    else:
        row_text = [td.get_text(strip=True) for td in row.find_all(['th', 'td'])]
        row_text[0] = f"{team_abbrev}_{row_text[0]}"
        team_stat_rows.append(row_text)

In [8]:
df_team_stats = pd.DataFrame(columns=team_stat_rows[0], data=team_stat_rows[1:])

In [9]:
df_team_stats["Att_passing_totals"] = df_team_stats["Att_passing_totals"].astype(int)
df_team_stats["Att_rushing_totals"] = df_team_stats["Att_rushing_totals"].astype(int)

In [10]:
df_team_stats["total_snaps"] = df_team_stats["Att_passing_totals"] + df_team_stats["Att_rushing_totals"]

In [11]:
df_team_stats

,Player,PF_totals,Yds_totals,Ply_totals,Y/P_totals,TO_totals,FL_totals,1stD_totals,Cmp_passing_totals,Att_passing_totals,...,1stPy_penalties,#Dr_drives,Sc%_drives,TO%_drives,Start_average_drive,Time_average_drive,Plays_average_drive,Yds_average_drive,Pts_average_drive,total_snaps
0,dal_Team Stats,467,6034,1114,5.4,23,5,347,355,556,...,24,195,41.5,10.8,Own 29.0,2:35,5.88,30.9,2.31,1087
1,dal_Opp. Stats,342,5613,1101,5.1,33,17,327,345,550,...,34,197,33.5,16.2,Own 29.2,2:40,5.8,28.4,1.65,1047
2,dal_Lg Rank Offense,4,11,,,17,2,11,,19,...,,,6,20,13,27,18,18,7,25
3,dal_Lg Rank Defense,5,12,,,1,1,14,,11,...,,,25,1,27,5,8,6,4,38


In [12]:
df_team_stats[["Att_passing_totals", "Att_rushing_totals", "total_snaps"]]

,Att_passing_totals,Att_rushing_totals,total_snaps
0,556,531,1087
1,550,497,1047
2,19,6,25
3,11,27,38


# Building Game df

In [16]:
game_table = table_list[1]
rows = game_table.find_all("tr")

game_rows = list()
for i, row in enumerate(rows[1:]):
    if i == 0:
        header_text = [td.get_text(strip=True) for td in row.find_all(['th', 'td'])]
        header_text[3] = "Game Time"
        header_text[4] = "Link to Boxscore"
        header_text[5] = "Result"
        header_text[8] = "Home/Away"
        
        game_rows.append(header_text)
    else:
        row_text = [td.get_text(strip=True) for td in row.find_all(['th', 'td'])]
        game_rows.append(row_text)

In [17]:
df_game = pd.DataFrame(columns=game_rows[0], data=game_rows[1:])

In [18]:
df_game.head()

,Week,Day,Date,Game Time,Link to Boxscore,Result,OT,Rec,Home/Away,Opp,...,RushY,TO,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Sp. Tms
0,1,Sun,September 11,8:20PM ET,boxscore,L,,0-1,,Tampa Bay Buccaneers,...,71,1,18,347,195,152,1,-19.40,-0.55,2.24
1,2,Sun,September 18,4:25PM ET,boxscore,W,,1-1,,Cincinnati Bengals,...,107,1,19,254,165,89,,6.08,-0.76,-1.25
2,3,Mon,September 26,8:15PM ET,boxscore,W,,2-1,@,New York Giants,...,176,,22,336,169,167,1,10.78,0.51,-0.01
3,4,Sun,October 2,1:00PM ET,boxscore,W,,3-1,,Washington Commanders,...,62,,17,297,155,142,2,4.19,13.42,-3.30
4,5,Sun,October 9,4:25PM ET,boxscore,W,,4-1,@,Los Angeles Rams,...,163,,14,323,285,38,3,-8.44,11.11,6.98


# Team Dict Translation

In [ ]:
teams = ['Browns', 'Dolphins', 'Colts', 'Buccaneers', 'Bears', 'Patriots',
       'Cowboys', 'Chiefs', 'Vikings', 'Redskins', 'Chargers', 'Lions',
       'Broncos', 'Giants', 'Texans', 'Saints', 'Ravens', 'Steelers',
       'Falcons', 'Jets', 'Eagles', 'Seahawks', 'Packers', 'Jaguars',
       'Bengals', 'Cardinals', 'Panthers', '49ers', 'Bills', 'Raiders',
       'Rams', 'Titans', 'Commanders']

In [ ]:
teams.sort()

In [ ]:
for team in teams:
    print(f"'{team}': ,")